In [2]:
from scipy.stats import norm
import numpy as np

# 给定的参数
S0 = 5.35  # 当前股票价格
K = 5.65  # 行权价格
r = 0.054  # 无风险利率
T = 0.75  # 到期时间，9个月
sigma = 0.3  # 股票波动率

# 计算 d1 和 d2
d1 = (np.log(S0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

# 使用Black-Scholes公式计算期权价格
C = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
C


0.5198141150337765

In [10]:
def exotic_option_price_monte_carlo(S0, r, T, sigma, M):
    dt = T  # 直接计算到期时的股票价格
    a = r - 0.5 * sigma ** 2
    option_values = np.zeros(M)

    for i in range(M):
        z = np.random.normal()  # 标准正态分布随机数
        ST = S0 * np.exp(a * T + sigma * z * np.sqrt(T))  # 计算到期时的股票价格
        payoff = max(8 * np.cos(ST) - 5.65, 0)  # 计算期权的回报
        option_values[i] = payoff

    # 计算期权的平均回报并折现
    option_price = np.exp(-r * T) * np.mean(option_values)
    return option_price

# 参数
M_large = 100000  # 较大的模拟次数

# 计算外来期权的价格
exotic_option_price = exotic_option_price_monte_carlo(S0, r, T, sigma, M_large)
exotic_option_price


0.42805425558874416

In [5]:
# 参数设定
S0 = 5.35  # 当前股票价格
K = 5.65   # 行权价格
r = 0.054  # 无风险利率
T = 1      # 到期时间，1年
sigma = 0.3  # 股票波动率

# 计算亚洲期权的特定参数
sigma_G = sigma / np.sqrt(3)  # 调整后的波动率
b = 0.5 * (r - sigma_G ** 2 / 2)  # 中间变量b
d1 = (np.log(S0 / K) + (b + sigma_G ** 2 / 2) * T) / (sigma_G * np.sqrt(T))  # d1
d2 = d1 - sigma_G * np.sqrt(T)  # d2

# 使用特定的Black-Scholes公式计算亚洲期权的价格
asian_option_price = S0 * np.exp((b - r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
asian_option_price


0.27824333946938884

In [9]:
sigma_G,b,d1,d2

(0.17320508075688773, 0.0195, -0.11581059956658811, -0.28901568032347585)

In [14]:
def asian_call_option_monte_carlo(S0, K, r, T, sigma, M, n):
    dt = T / n  # 小周期长度
    call_values = np.zeros(M)  # 存储每次模拟的期权价值

    for i in range(M):
        log_sum = 0  # 存储 log(S_t) 的和
        S = S0

        # 模拟股票价格并计算 log(S_t) 的和
        for _ in range(n):
            S *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.random.normal() * np.sqrt(dt))
            log_sum += np.log(S)

        # 计算几何平均价格
        A_T = np.exp(log_sum / n)

        # 计算期权价值
        call_values[i] = max(A_T - K, 0)

    # 计算期权的平均收益并折现
    call_price = np.exp(-r * T) * np.mean(call_values)
    return call_price

# 参数设定
ks = [3, 4, 5, 6, 7]  # 由于计算量大，首先计算 k=3 和 k=4 的情况
asian_option_prices_mc = {f'10^{k}': asian_call_option_monte_carlo_efficient(S0, K, r, T, sigma, 10**k, 100) for k in ks}
asian_option_prices_mc


{'10^3': 0.29514153180208147,
 '10^4': 0.2844820290295809,
 '10^5': 0.281441776904544,
 '10^6': 0.2813120866602236,
 '10^7': 0.281438652105844}

In [12]:
def asian_call_option_monte_carlo_efficient(S0, K, r, T, sigma, M, n):
    dt = T / n  # 小周期长度
    t = np.linspace(dt, T, n)  # 时间点

    # 生成所有路径的随机扰动
    Z = np.random.normal(0, np.sqrt(dt), (n, M))

    # 计算每个时间点的股票价格
    S = S0 * np.exp(np.cumsum((r - 0.5 * sigma ** 2) * dt + sigma * Z, axis=0))

    # 计算几何平均价格
    A_T = np.exp(np.mean(np.log(S), axis=0))

    # 计算期权的收益
    call_values = np.maximum(A_T - K, 0)

    # 计算期权的平均收益并折现
    call_price = np.exp(-r * T) * np.mean(call_values)
    return call_price

# 参数设定
M_small = 10**3  # 较小的模拟次数
asian_option_price_mc_efficient = asian_call_option_monte_carlo_efficient(S0, K, r, T, sigma, M_small, 100)
asian_option_price_mc_efficient


0.2931072073020639